In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import calendar 
from datetime import datetime

In [2]:
df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
DataAcc

,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
...,...,...,...,...
2020-12-31 21:55:00,2020S019513,19.0,VICÁLVARO,Atropello a persona
2020-12-31 22:00:00,2020S019516,12.0,USERA,Choque contra obstáculo fijo
2020-12-31 22:25:00,2020S019517,13.0,PUENTE DE VALLECAS,Choque contra obstáculo fijo


In [3]:
DataAcc = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8) | (DataAcc['cod_distrito'] == 5) | (DataAcc['cod_distrito'] == 7))# | (DataAcc['cod_distrito'] == 7) | (DataAcc['cod_distrito'] == 1) | (DataAcc['cod_distrito'] == 4))]  
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro') #& (DataAcc['tipo_accidente'] != 'Vuelco')
                        & (DataAcc['tipo_accidente'] != 'Atropello a animal') & (DataAcc['tipo_accidente'] != 'Colisión frontal')
                         & (DataAcc['tipo_accidente'] != 'Solo salida de la vía'))]
DataAcc['Accidente'] = True
DataAcc = DataAcc.drop_duplicates(subset = ['num_expediente'])

/tmp/ipykernel_9313/4244713018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataAcc['Accidente'] = True


In [4]:
DataAcc = DataAcc.reset_index()

In [5]:
#Patrón Pardo: Hora de accidente es una hora antes y afEcta la hora siguiente si y solo si el minuto de la hora resultante es >=10
#Patrón Hortaleza: Accidente tiene efecto una hora después si y solo si el minuto de la hora es <10, accidente por alcance no tienen ningún efecto y accidente en la noche no tiene efecto (23:00)
#Patrón Chamartín: La hora de accidente real es la hora registrada pero afecta a la hora siguiente, accidente en la madrugada no tiene efecto 
def ModTime(ds):
    global valor
    global fila  
    global prueba
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute    
    dia = date.day
    mes = date.month    
    consulta = DataAcc.loc[fila]    
    distrito = consulta['distrito']        
    
    if (distrito == 'FUENCARRAL-EL PARDO'):
        try:
            if (hora < 23) & (minuto >= 0):
                date = date.replace(hour=hora+1,minute=0)                        
            elif (hora == 23) & (minuto >= 0):
                date = date.replace(day=dia+1,hour=0,minute=0)                            
        except:            
            date = date.replace(month=mes+1, day=1, hour=0, minute=0)  
            
    elif (distrito == 'HORTALEZA'):
        try:
            if (hora < 23) & (minuto == 0):
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 23) & (minuto <= 45): ############REVISAR
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto <= 45):
                date = date.replace(day=dia+1,hour=0,minute=0)    
            elif (hora < 22) & (minuto > 45):        
                date = date.replace(hour=hora+2,minute=0)         
            elif (hora == 22) & (minuto > 45):
                date = date.replace(day=dia+1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 45):
                date = date.replace(day=dia+1,hour=1,minute=0)
        except:               
            if (hora == 23) & (minuto == 0):
                date = date.replace(month=mes+1,day=1,hour=0)            
            elif (hora == 23) & (minuto <= 45):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)                                
            elif (hora == 22) & (minuto > 45):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 45):
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)              
                
    elif (distrito == 'CHAMARTÍN'):           
        try:
            if (hora < 23) & (minuto == 0):                
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):                
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 22) & (minuto != 0):               
                date = date.replace(hour=hora+2,minute=0)            
            elif (hora == 22) & (minuto != 0):                
                date = date.replace(day=dia+1,hour=0,minute=0)                
            elif (hora == 23) & (minuto != 0):                
                date = date.replace(day=dia+1,hour=1,minute=0)                            
        except:            
            if (hora == 23) & (minuto == 0):                
                date = date.replace(month=mes+1,day=1,hour=0)                        
            elif (hora == 22) & (minuto != 0):                
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto != 0):                
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)    
                
    elif (distrito == 'CHAMBERÍ'):           
        try:
            if (minuto == 0):                
                date = date.replace(hour=hora,minute=0)            
            elif (minuto <= 10 ):               
                date = date.replace(hour=hora,minute=0)                       
            elif (hora < 23) & (minuto > 10):                
                date = date.replace(hour=hora+1,minute=0)                     
            elif (hora == 23) & (minuto > 10):                
                date = date.replace(day=dia+1,hour=0,minute=0)            
        except:            
            if (hora == 23):                
                date = date.replace(month=mes+1,day=1,hour=0)                        
    fila += 1
    
    return (date)
prueba=0
fila = 0
DataAcc['ds1'] = DataAcc['ds'].apply(ModTime)

# Agregamos columna ds2 que indica el aumento de los desplazamientos que ocurre 1 hora después de un accidente.

In [6]:
def ds2(ds):
    date = pd.to_datetime(ds)
    año = date.year
    mes = date.month
    dia = date.day
    hora = date.hour    
    try:
        if (hora < 23):   # Revisar si esta bien esta condición
            date = date.replace(hour=hora+1)        
        elif (hora == 23):
            date = date.replace(day=dia+1,hour=0)
    except:        
        if (mes < 12):
            date = date.replace(month=mes+1, day=1, hour=0)         
        else:
            date = date.replace(year=año+1, month=1, day=1, hour=0)
            
    return (date)

DataAcc['ds2'] = DataAcc['ds1'].apply(ds2)

In [7]:
def Contador(ds):    
#     lista = Acc_ds1['ds1'].tolist()   
    lista = DataAcc['ds1'].tolist()   
    NumAcc = lista.count(ds)    
    return (NumAcc)
DataAcc['NumAcc'] = DataAcc['ds1'].apply(Contador)

In [8]:
def AjusteDs2(NumAcc):
    global valor
    global fila          
    if fila != 0:
        consulta = Acc_ds2.loc[fila-1]           
        dsBack = consulta['ds2']
        NumAccBack = consulta['NumAcc']            
        if NumAccBack > 2:        
            match = dsBack in Acc_ds1.ds1.values       
            if match == True:             
                valor = NumAccBack
            else:
                valor = NumAcc
        else:
            valor = NumAcc
    else:
        valor = NumAcc
    fila+=1
    return valor

In [9]:
DataAcc = DataAcc.set_index('ds')

In [10]:
Prueba = DataAcc.loc['2020-07-06':'2020-07-06']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2,NumAcc
ds,,,,,,,,
2020-07-06 10:45:00,2020S009407,7.0,CHAMBERÍ,Atropello a persona,True,2020-07-06 11:00:00,2020-07-06 12:00:00,1
2020-07-06 12:00:00,2020S009461,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 13:00:00,2020-07-06 14:00:00,2
2020-07-06 12:08:00,2020S009507,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-07-06 13:00:00,2020-07-06 14:00:00,2
2020-07-06 17:05:00,2020S009427,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True,2020-07-06 18:00:00,2020-07-06 19:00:00,1
2020-07-06 19:22:00,2020S009472,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 21:00:00,2020-07-06 22:00:00,1
2020-07-06 21:10:00,2020S009536,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True,2020-07-06 22:00:00,2020-07-06 23:00:00,1


In [11]:
Prueba = DataAcc.loc['2020-07-13':'2020-07-13']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2,NumAcc
ds,,,,,,,,
2020-07-13 13:45:00,2020S009778,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-13 14:00:00,2020-07-13 15:00:00,1
2020-07-13 14:35:00,2020S009752,7.0,CHAMBERÍ,Alcance,True,2020-07-13 15:00:00,2020-07-13 16:00:00,1
2020-07-13 15:10:00,2020S009801,5.0,CHAMARTÍN,Alcance,True,2020-07-13 17:00:00,2020-07-13 18:00:00,2
2020-07-13 15:40:00,2020S009797,5.0,CHAMARTÍN,Colisión lateral,True,2020-07-13 17:00:00,2020-07-13 18:00:00,2
2020-07-13 19:40:00,2020S009798,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,True,2020-07-13 20:00:00,2020-07-13 21:00:00,1
2020-07-13 20:18:00,2020S009764,7.0,CHAMBERÍ,Colisión lateral,True,2020-07-13 21:00:00,2020-07-13 22:00:00,1


In [12]:
Prueba = DataAcc.loc['2020-07-19':'2020-07-19']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2,NumAcc
ds,,,,,,,,
2020-07-19 15:30:00,2020S010182,8.0,FUENCARRAL-EL PARDO,Alcance,True,2020-07-19 16:00:00,2020-07-19 17:00:00,1
2020-07-19 23:30:00,2020S010094,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,True,2020-07-20 00:00:00,2020-07-20 01:00:00,1


In [13]:
Prueba = DataAcc.loc['2020-07-20':'2020-07-20']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2,NumAcc
ds,,,,,,,,
2020-07-20 06:00:00,2020S010098,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-07-20 07:00:00,2020-07-20 08:00:00,1
2020-07-20 09:15:00,2020S010113,7.0,CHAMBERÍ,Alcance,True,2020-07-20 10:00:00,2020-07-20 11:00:00,1
2020-07-20 12:10:00,2020S010159,5.0,CHAMARTÍN,Colisión fronto-lateral,True,2020-07-20 14:00:00,2020-07-20 15:00:00,3
2020-07-20 12:55:00,2020S010130,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-07-20 14:00:00,2020-07-20 15:00:00,3
2020-07-20 13:30:00,2020S010155,7.0,CHAMBERÍ,Alcance,True,2020-07-20 14:00:00,2020-07-20 15:00:00,3
2020-07-20 14:48:00,2020S010122,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-20 15:00:00,2020-07-20 16:00:00,1
2020-07-20 17:05:00,2020S010174,5.0,CHAMARTÍN,Colisión fronto-lateral,True,2020-07-20 19:00:00,2020-07-20 20:00:00,1
2020-07-20 18:45:00,2020S010175,5.0,CHAMARTÍN,Alcance,True,2020-07-20 20:00:00,2020-07-20 21:00:00,2
2020-07-20 19:30:00,2020S010185,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,True,2020-07-20 20:00:00,2020-07-20 21:00:00,2


In [14]:
Prueba = DataAcc.loc['2020-07-26':'2020-07-26']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2,NumAcc
ds,,,,,,,,
2020-07-26 16:27:00,2020S010439,8.0,FUENCARRAL-EL PARDO,Colisión lateral,True,2020-07-26 17:00:00,2020-07-26 18:00:00,1
2020-07-26 18:20:00,2020S010481,8.0,FUENCARRAL-EL PARDO,Colisión lateral,True,2020-07-26 19:00:00,2020-07-26 20:00:00,1
2020-07-26 22:00:00,2020S010490,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-07-26 23:00:00,2020-07-27 00:00:00,1
2020-07-26 23:00:00,2020S010486,8.0,FUENCARRAL-EL PARDO,Colisión fronto-lateral,True,2020-07-27 00:00:00,2020-07-27 01:00:00,1


In [15]:
Prueba = DataAcc.loc['2020-07-27':'2020-07-27']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2,NumAcc
ds,,,,,,,,
2020-07-27 02:45:00,2020S010458,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-27 04:00:00,2020-07-27 05:00:00,1
2020-07-27 09:10:00,2020S011678,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-27 11:00:00,2020-07-27 12:00:00,1
2020-07-27 09:25:00,2020S010466,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-27 10:00:00,2020-07-27 11:00:00,1
2020-07-27 13:10:00,2020S010512,7.0,CHAMBERÍ,Alcance,True,2020-07-27 13:00:00,2020-07-27 14:00:00,1
2020-07-27 15:30:00,2020S010477,7.0,CHAMBERÍ,Colisión fronto-lateral,True,2020-07-27 16:00:00,2020-07-27 17:00:00,1
2020-07-27 20:30:00,2020S010542,5.0,CHAMARTÍN,Alcance,True,2020-07-27 22:00:00,2020-07-27 23:00:00,4
2020-07-27 21:40:00,2020S010536,8.0,FUENCARRAL-EL PARDO,Alcance,True,2020-07-27 22:00:00,2020-07-27 23:00:00,4
2020-07-27 21:45:00,2020S010544,16.0,HORTALEZA,Alcance,True,2020-07-27 22:00:00,2020-07-27 23:00:00,4
2020-07-27 21:45:00,2020S010538,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-07-27 22:00:00,2020-07-27 23:00:00,4


In [16]:
# Prueba = DataAcc.loc['2020-07-21':'2020-07-21']
# Prueba

In [17]:
# Prueba = DataAcc.loc['2020-07-28':'2020-07-28']
# Prueba

In [18]:
DataAcc = DataAcc.reset_index()

In [19]:
DataAcc = DataAcc.set_index('ds')
Acc_ds = DataAcc.loc[:,['Accidente']]
DataAcc = DataAcc.set_index('ds1')
Acc_ds1 = DataAcc.loc[:,['Accidente','NumAcc']]#,'distrito'
DataAcc = DataAcc.set_index('ds2')
Acc_ds2 = DataAcc.loc[:,['Accidente','NumAcc']]#,'distrito'

Acc_ds1 = Acc_ds1.reset_index() 
Acc_ds1 = Acc_ds1.drop_duplicates(subset = ['ds1'])
Acc_ds1 = Acc_ds1.set_index('ds1') 

Acc_ds2 = Acc_ds2.reset_index() 
Acc_ds2 = Acc_ds2.drop_duplicates(subset = ['ds2'])
Acc_ds2 = Acc_ds2.set_index('ds2') 

Acc_ds2 = Acc_ds2.reset_index() 
Acc_ds1 = Acc_ds1.reset_index() 
fila = 0
Acc_ds2['NumAcc'] = Acc_ds2['NumAcc'].apply(AjusteDs2)

Acc_ds1 = Acc_ds1.set_index('ds1') 
Acc_ds2 = Acc_ds2.set_index('ds2') 

In [20]:
Acc_ds1
Data=Acc_ds1.loc['2020-07-28':'2020-07-28']
Data

/tmp/ipykernel_9313/257229665.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  Data=Acc_ds1.loc['2020-07-28':'2020-07-28']


,Accidente,NumAcc
ds1,,
2020-07-28 09:00:00,True,1
2020-07-28 10:00:00,True,1
2020-07-28 11:00:00,True,1
2020-07-28 14:00:00,True,1
2020-07-28 16:00:00,True,1
2020-07-28 18:00:00,True,1
2020-07-28 20:00:00,True,1
2020-07-28 21:00:00,True,1
2020-07-28 22:00:00,True,1


In [21]:
Data=Acc_ds2.loc['2020-07-27':'2020-07-27']
Data

,Accidente,NumAcc
ds2,,
2020-07-27 00:00:00,True,1
2020-07-27 01:00:00,True,1
2020-07-27 05:00:00,True,1
2020-07-27 12:00:00,True,1
2020-07-27 11:00:00,True,1
2020-07-27 14:00:00,True,1
2020-07-27 17:00:00,True,1
2020-07-27 23:00:00,True,4


In [22]:
Data=Acc_ds1.loc['2020-07-22':'2020-07-22']
Data

/tmp/ipykernel_9313/2299814672.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  Data=Acc_ds1.loc['2020-07-22':'2020-07-22']


,Accidente,NumAcc
ds1,,
2020-07-22 11:00:00,True,1
2020-07-22 14:00:00,True,1
2020-07-22 13:00:00,True,1
2020-07-22 15:00:00,True,1
2020-07-22 17:00:00,True,1
2020-07-22 22:00:00,True,1
2020-07-22 23:00:00,True,1


In [23]:
Data=Acc_ds2.loc['2020-07-22':'2020-07-22']
Data

/tmp/ipykernel_9313/742982244.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  Data=Acc_ds2.loc['2020-07-22':'2020-07-22']


,Accidente,NumAcc
ds2,,
2020-07-22 12:00:00,True,1
2020-07-22 15:00:00,True,1
2020-07-22 14:00:00,True,1
2020-07-22 16:00:00,True,1
2020-07-22 18:00:00,True,1
2020-07-22 23:00:00,True,1


In [24]:
# Acc_ds.to_csv("Acc2_ds.csv")
# Acc_ds1.to_csv("Acc2_ds1.csv")
# Acc_ds2.to_csv("Acc2_ds2.csv")

In [25]:
print(Acc_ds1)
print(Acc_ds2)

                     Accidente  NumAcc
ds1                                   
2020-01-01 02:00:00       True       2
2020-01-01 10:00:00       True       1
2020-01-01 13:00:00       True       1
2020-01-01 15:00:00       True       1
2020-01-01 17:00:00       True       1
...                        ...     ...
2020-12-31 15:00:00       True       1
2020-12-31 16:00:00       True       2
2020-12-31 18:00:00       True       3
2020-12-31 20:00:00       True       2
2020-12-31 22:00:00       True       2

[2010 rows x 2 columns]
                     Accidente  NumAcc
ds2                                   
2020-01-01 03:00:00       True       2
2020-01-01 11:00:00       True       1
2020-01-01 14:00:00       True       1
2020-01-01 16:00:00       True       1
2020-01-01 18:00:00       True       1
...                        ...     ...
2020-12-31 16:00:00       True       1
2020-12-31 17:00:00       True       2
2020-12-31 19:00:00       True       3
2020-12-31 21:00:00       True       2
